The data that has been wrangled is data from https://mapzen.com/data/metro-extracts/metro/jakarta_indonesia/

I originate from Indonesia so I thought it would be nice if I can explore the data that I should be familiar with. Thus, I can find the most common errors in data particularly in my area.

# Problems Encountered in the Map

There are several issues found in the data, mostly came from the `tag` tag.

1. Inconsistent address format, some only including address, residencial cluster,
```
\xc3\xa2\xc2\x80\xc2\x8eJl. Ir. H. Djuanda No. 95'
JI.Margonda Raya No. 428, Beji, Depok , Indonesia'
'M.I. RIDWAN RAIS NO. 37, Beji Timur. Depok'
'Sentra Niaga Puri Indah'
'Pamulang Permai blok D III no. 1-2',
'22'
```

2. Wrong formatted postal codes
```
b'16127.'
b'14450.'
b'11550.'
b'\xc3\xa2\xc2\x80\xc2\x8e15414'
b'Lippo Karawaci 1600 Tangerang 15811'
b'151416'
```

3. Inconsistent language used on city
```
'Jakarta Selatan',
'South Jakarta',
```

4. Incosistent Phone number
```
'+62 21 799 0888',
'+62 21 5263137',
'14045',
'622178834966',
'+62 8983 2943',
'(0251) 831 6348',
```

Thus, before the data exported into the database, these errors should be cleaned. Below are the logic behind the data cleaning respect to each error.

1. To clean the in consistent address format, in this case the street (Jalan) naming, following algorithm conducted:
    - Replace all abbreviated "Jalan" such as jl, jln, Jln., and so on.
    - Delete all double spaces such as "Jalan  A"
    - Remove all non ASCII character in the street name such as "\xe2\x80\x8eJalan Ir. H. Djuanda No. 95" (mostly we do not use any latin/utf characters. However, we need more research on this)
    - Remove all non address and its number from the text, such as city name and country

    Thus, below is the code function:

    ```
    ADDRESS_ABBRV = re.compile(r'(j|J)(l|ln|I|L|LN)(\s|\.)|jalan')
    
    def fix_address(data):
        data = ADDRESS_ABBRV.sub('Jalan ', data)
        data = data.replace("  ", " ") #delete double spaces
        data = data.encode('ascii', 'ignore').decode() #remove unicode characters
        if 'Jalan' not in data:
            data = "Jalan "+ data
        for value in re.split(',\s|,', data):
            if "Jalan" in value:
                data = value
        return data
    ```
    
2. To clean the postal code is quite simple, it only need a regex as follows:
    ```
    POSTAL_CODE = re.compile(r'[0-9]{5}')
    
    def fix_postal(data):
    if len(POSTAL_CODE.findall(data)) > 0:
        return POSTAL_CODE.findall(data)[0]
    else:
        return '00000'
    ```

3. It interesting in Indonesia sometimes we confused when to use english or bahasa. Thus, the inconsistent language may occurs in city name such as South (Selatan), to "simple" fix the issue, I have provided a simple dictionary as follows to tranlate all to bahasa: 
    ```
    CITY_TRANSLATION = {
        "south jakarta": "Jakarta Selatan",
        "north jakarta": "Jakarta Utara",
        "west jakarta": "Jakarta Barat",
        "east jakarta": "Jakarta Timur",
        "central jakarta": "Jakarta Pusat"
    }
    ```
    However, it is just a simple fix, further development such as per-word/context translation may help this better.

4. Generally every people have their own format for their phone number, as as "simple" fix, I remove all the special character in phone number to make it more consistent.
  


# Overview of the Data
As a general overview, there are 16071868 lines that were processed resulting following files.
 ```
    jakarta_indonesia.osm: 3.04 GB
    nodes.csv: 1.45 GB
    nodes_tags.csv: 17.9 MB
    ways.csv: 253 MB
    ways_nodes.csv: 507.4 MB 
    ways_tags.csv: 703.5 MB
    pythonsqlite.db: 2.13 GB
    ```

### Number of Unique Users
```
sqlite> SELECT COUNT(DISTINCT(com.uid)) FROM (SELECT uid FROM node UNION ALL SELECT uid FROM way) com;

2467
```


### Number of Nodes and Ways
```
sqlite> SELECT COUNT(*) FROM node;

12994948

sqlite> SELECT COUNT(*) FROM way;

3076920
```

### Top Contributed Users
```
sqlite> SELECT uid, user, COUNT(uid) AS cnt FROM node WHERE uid IN (SELECT uid FROM node UNION ALL SELECT uid FROM way) GROUP BY uid ORDER BY cnt DESC LIMIT 10;

4159996|Akrimullah|970757
5539627|Yeni Primasari|471231
2901733|adiatmad|457934
4706330|endang_p|397020
5520501|Dion Qairawan|389632
5520505|Hanif Arafah Mustofa|362426
1840603|martinmbaihaqi|353253
538459|Alex Rollin|349773
5454005|Riyadi Wibowo|326414
136807|Farras|323898
```

### Number of Top 10 City
```
sqlite> SELECT value, COUNT(value) FROM node_tags WHERE key = 'city' GROUP BY value ORDER BY COUNT(value) DESC LIMIT 10;

DKI Jakarta|15208
Jakarta Selatan|964
Jakarta Pusat|528
Jakarta Barat|504
Bekasi|412
Jakarta Utara|280
Jakarta Timur|272
Banten|220
Depok|188
Tangerang|108
```

It is interesting that DKI Jakarta can be considered either as Jakarta Selatan, Jakarta Pusat, Jakarta Barat, Jakarta Utara, and Jakarta Timur because it is a special region, DKI Jakarta sometimes misinterpreted as a city, in fact it is a province.


# Other ideas about the dataset

### Issue and Suggestion
The main issue faced on the dataset is the size of the data, it took very long time and too much information that can hinder exploration efficiency. To overcome the issue, I explore the file with limited number of lines. For example, only 200000 out of 16071868 lines were explored to find ill presented data. However, this method will leave other potential problems in the data that never be explored (can be seen in Number of Top 10 City, if I not limit the result, it will show more error such as West Java, it should be a province not a city). Thus, it is best to use more specific/smaller data area to be explored as it may reduce load of computing tasks and time. 

Another suggestion, to reduce inconsistent data values, OSM can implement value suggestion based on previous data or implement token normalisasion based on dictionary of previous correct data. However, this approach may reduce flexibility of new terms, such as address or buildings. Thus it should be very careful where to implement this suggestion, a good implementation can be applied to City, Province, or Postal Code value.

### Other Experiment on the Data
#### Top 10 Key Used and the User
```
sqlite> SELECT user, key, COUNT(key) cnt FROM node, node_tags WHERE node.id = node_tags.id GROUP BY user, key ORDER BY cnt DESC LIMIT 10;

   User     |    Key     | Count
   
Firman Hadi   created_by   45436
Alex Rollin   natural      20780
adiatmad      source       17836
Alex Rollin   type         6900
ceyockey      source       5004
gafuri        natural      3700
ceyockey      natural      3324
jeffhaack     country      3304
jeffhaack     name         3304
jeffhaack     place        3304
```
It is interesting that the highest key is is not inputted by the highest contributor. Further experiment to find what keys that inputted by the top contributor.

# Conclusion

There are many issues found on the OSM data. It might be because its nature as open source platform that promotes flexibility. However, the most common drawback is inconsistent values, typo, etc. Furthermore, the data that explored in this project is very big (2.13 GB in total in the db) which lead to unadequate time for exploration. Thus, smaller dataset is more favourable. 

